In [1]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds #tfds contains many more datasets besides MNIST to explore and use

In [2]:
# with_info=True provides tuple with some basic information 
# as_supervised=True loads the data into a 2-tuple structure(input,target)

mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True) 


In [3]:
# MNIST dataset does not have validation set, hence it needs to be created from training set(its the bigger one)

mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

In [4]:
# We want to scale the data in such a way that it is more numerically stable such as all values should be between 0 and 1

def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255. # as each pixel is between 0 and 255, dividing by 255 will make all values be somewhere between 0 and 1
    return image, label

In [5]:
# Now that we have a scaling function the dataset needs to be transformed

scaled_train_and_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)

In [6]:
# Now the dataset should be shuffled as it may have been sorted or in a certain order
# Not shuffling might lead to wrong interpretations as each bach might just ressemble only one number

BUFFER_SIZE = 10000 # When dealing with big datasets, not all data can be put into memory and needs to be buffered partially

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)



In [7]:
# As we want to use the SGD we need to split our dataset into different batches

BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE) # Tells the model how many samples it should take into each batch
validation_data = validation_data.batch(num_validation_samples) # As we don't backpropagate for the validation set we can use the hole set to calculate the exact values
test_data = test_data.batch(num_test_samples)

In [8]:
# As our dataset is in a 2-tuple format we must extract and convert the validation inputs and targets appropriately

validation_inputs, validation_targets = next(iter(validation_data))


# Model

In [9]:
input_size = 784 # Our input is 28x28x1
output_size = 10
hidden_layer_size = 800 # Each hidden layer can have its own width, might lead to better result, feel free to play around with

model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28,28,1)),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # First hidden layer
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),  # Second hidden layer
                            tf.keras.layers.Dense(output_size, activation='softmax') # Output layer
                            ])


In [10]:
# Model needs to be compiled

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy']) # Accuracy calculated throughout each epoch

In [11]:
# Next the model needs to be trained

NUM_EPOCHS = 5 # Sets the number of epochs we will do

model.fit(train_data, epochs = NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose=2, validation_steps=10) # verbose=2 makes sure only the most important information is received per epoch


Epoch 1/5
540/540 - 28s - loss: 0.2013 - accuracy: 0.9387 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/5
540/540 - 26s - loss: 0.0764 - accuracy: 0.9767 - val_loss: 0.0828 - val_accuracy: 0.9755
Epoch 3/5
540/540 - 26s - loss: 0.0513 - accuracy: 0.9839 - val_loss: 0.0707 - val_accuracy: 0.9797
Epoch 4/5
540/540 - 26s - loss: 0.0387 - accuracy: 0.9873 - val_loss: 0.0546 - val_accuracy: 0.9833
Epoch 5/5
540/540 - 26s - loss: 0.0329 - accuracy: 0.9890 - val_loss: 0.0370 - val_accuracy: 0.9877


In [12]:
# Last but not least the model needs to be used on the test data
# The hyperparameters have been adapted to the validation set hence possibly leading to overfitting to the validation set
# The test set is an entirely unknown set and after testing the model should not be adjusted in order to prevent fitting the test set

test_loss, test_accuracy = model.evaluate(test_data)

# Getting a test accuracy close to val_accuracy shows that the model has not been overfitted

print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.)) 

1/1 [==============================] - 4s 4s/step - loss: 0.0738 - accuracy: 0.9781
Test loss: 0.07. Test accuracy: 97.81%
